In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from  torchvision.transforms import Grayscale
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18, ResNet18_Weights


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/challenges-in-representation-learning-facial-expression-recognition-challenge/train.csv")

In [ ]:
data

,emotion,pixels
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...
...,...,...
28704,2,84 85 85 85 85 85 85 85 86 86 86 87 86 86 91 9...
28705,0,114 112 113 113 111 111 112 113 115 113 114 11...
28706,4,74 81 87 89 95 100 98 93 105 120 127 133 146 1...
28707,0,222 227 203 90 86 90 84 77 94 87 99 119 134 14...


In [ ]:
data['pixels'] = data['pixels'].apply(lambda x: np.array(list(map(float, x.split()))))

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(np.array(data['pixels'].values.tolist()), data['emotion'].values, test_size=0.2)
X_train, X_test, y_train, y_test = torch.tensor(X_train), torch.tensor(X_test), torch.tensor(y_train), torch.tensor(y_test)

In [ ]:
X_train = X_train.reshape(-1, 1, 48, 48)
X_test = X_test.reshape(-1, 1, 48, 48)

In [ ]:
X_train = torch.cat([X_train, X_train, X_train], dim=1)
X_test = torch.cat([X_test, X_test, X_test], dim=1)

In [ ]:
# upsample = nn.Upsample(scale_factor=5.09, mode="nearest")

# X_train = upsample(X_train).float()
# X_test = upsample(X_test).float()

In [ ]:
class CustomTensorDataset(Dataset): 
  def __init__(self, tensors, transform=None): 
    self.tensors = tensors 
    self.transform = transform 
  
  def __len__(self): 
    return self.tensors[0].shape[0]
  
  def __getitem__(self, idx): 
    x = self.tensors[0][idx]
    if self.transform: 
      x_reshaped = x.reshape(1, 48, 48)
      x = self.transform(torch.cat([x_reshaped, x_reshaped, x_reshaped], dim=0))
    y = self.tensors[1][idx]

    return x, y

In [ ]:
train_tensor = CustomTensorDataset(tensors=(X_train, y_train), transform=None) 
train_dataloader = DataLoader(dataset=train_tensor, batch_size=64, shuffle=True)

test_tensor = CustomTensorDataset(tensors=(X_test, y_test), transform=None) 
test_dataloader = DataLoader(dataset=test_tensor, batch_size=64, shuffle=True)

In [ ]:
labels_map = {
    0: "Angry",
    1: "Disgust", 
    2: "Fear",
    3: "Happy", 
    4: "Sad", 
    5: "Suprise",
    6: "Neutral"
}

In [ ]:
train_features, train_labels = next(iter(train_dataloader))

In [ ]:
# figure = plt.figure(figsize=(10,10))
# rows, cols = 3, 3

# for i in range(1, cols*rows + 1): 
#   img = train_features[i].squeeze()
#   label = train_labels[i]
#   figure.add_subplot(rows, cols, i)
#   plt.title(f"{labels_map[label.item()]}")
#   plt.axis("off")
#   plt.imshow(img.reshape(48, 48), cmap="gray")
# plt.show()

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer): 
  size = len(dataloader.dataset)

  for batch, (X, y) in enumerate(dataloader): 
    # prediction & loss 
    X = X.float()
    pred = model(X)
    loss = loss_fn(pred, y)

    # backprop 
    optimizer.zero_grad()
    loss.backward() 
    optimizer.step()
    if batch % 100 == 0: 
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [ ]:
def test_loop(dataloader, model, loss_fn): 
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader: 
      X = X.float()
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")    

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"
weights = ResNet18_Weights.DEFAULT
model = resnet18(weights=weights)

for param in model.parameters(): 
  param.requires_grad = False 

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(labels_map))
model = model.to(device)

loss_fn = nn.CrossEntropyLoss()

lr = 0.001
momentum=0.9

optimizer_model = optim.SGD(model.fc.parameters(), lr=lr, momentum=momentum)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
# epochs=5
# for epoch in range(epochs):
#   print(f"Epoch {epoch+1}\n-------------------------------") 
#   train_loop(train_dataloader, model, loss_fn, optimizer_model)
#   test_loop(test_dataloader, model, loss_fn)
# print("Done!")

In [ ]:
class Bless1(nn.Module): 
  def __init__(self): 
    super(Bless1, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3))
    self.relu1 = nn.ReLU()
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3))
    self.relu2 = nn.ReLU()
    self.maxpool1 = nn.MaxPool2d(kernel_size=(2, 2))
    self.dropout1 = nn.Dropout(0.25)

    self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3))
    self.relu3 = nn.ReLU()
    self.maxpool2 = nn.MaxPool2d(kernel_size=(2, 2))
    self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3))
    self.relu4 = nn.ReLU()
    self.maxpool3 = nn.MaxPool2d(kernel_size=(2, 2))
    self.dropout2 = nn.Dropout(0.25)

    self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
    self.fc1 = nn.LazyLinear(1024)
    self.relu5 = nn.ReLU()
    self.dropout3 = nn.Dropout(0.5)
    self.fc2 = nn.Linear(1024, len(labels_map))

  def _forward_impl(self, x: torch.Tensor) -> torch.Tensor: 
    x = self.conv1(x)
    x = self.relu1(x)
    x = self.conv2(x)
    x = self.relu2(x)
    x = self.maxpool1(x)
    x = self.dropout1(x)

    x = self.conv3(x)
    x = self.relu3(x)
    x = self.maxpool2(x)
    x = self.conv4(x)
    x = self.relu4(x)
    x = self.maxpool3(x)
    x = self.dropout2(x)

    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = self.relu5(x)
    x = self.dropout3(x)
    x = self.fc2(x)
    return x 

  def forward(self, x): 
    return self._forward_impl(x)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Bless = Bless1()

for param in Bless.parameters(): 
  param.requires_grad = True 

loss_fn = nn.CrossEntropyLoss()

lr = 0.001
weight_decay=1e-6

optimizer_model = optim.Adam(Bless.parameters(), lr=lr, weight_decay=weight_decay)

In [ ]:
epochs=50
for epoch in range(epochs):
  print(f"Epoch {epoch+1}\n-------------------------------") 
  train_loop(train_dataloader, Bless, loss_fn, optimizer_model)
  test_loop(test_dataloader, Bless, loss_fn)
print("Done!")